In [5]:
class treeNode:
    def __init__(self,nameValue,numOccur,parentNode):
        self.name=nameValue
        self.count=numOccur
        self.nodeLink=None
        self.parent=parentNode
        self.children={}
    def inc(self,numOccur):
        self.count+=numOccur
    def display(self,ind=1):
        print('|        '*ind,self.name,' ',self.count)
        for child in self.children.values():
            child.display(ind+1)
def createTree(dataSet,minSupport):
    #计算所有1-项集出现的次数
    headerTable={}
    for transaction in dataSet:
        for item in transaction:
            headerTable[item]=headerTable.get(item,0)+dataSet[transaction]
    #移除小于支持度的1-项集
    for k in list(headerTable.keys()):
        if headerTable[k]<minSupport:
            del(headerTable[k])
    frequentItems=set(headerTable.keys())
    if len(frequentItems)==0: return None,None       ###没有1-项频繁集就返回None
    for key in list(headerTable):
        headerTable[key]=[headerTable[key],None]            ###列表第二项为指向第一个值的指针
    retTree=treeNode('Null set',1,None)
    for transaction,count in dataSet.items():
        #print(count,transaction)
        localD={}
        for item in transaction:
            if item in frequentItems:
                localD[item]=headerTable[item][0]
        if len(localD)>0:
            orderedTransaction=[v[0] for v in sorted(localD.items(),key=lambda x:x[1],reverse=True)]
            #print(orderedTransaction)
            updateTree(orderedTransaction,retTree,headerTable,count)
    
    return retTree,headerTable
    
def updateTree(transaction,inTree,headerTable,count):
    item0=transaction[0]
    if item0 in inTree.children:
        inTree.children[item0].inc(count)
    else:
        inTree.children[item0]=treeNode(item0,count,inTree)
        if headerTable[item0][1]==None:
            headerTable[item0][1]=inTree.children[item0]
        else:
            ###把已经存在的指针指向新建的节点
            updateHeader(headerTable[item0][1],inTree.children[item0])
    if len(transaction)>1:
        updateTree(transaction[1::],inTree.children[item0],headerTable,count)
def updateHeader(nodeToTest,targetNode):
    while(nodeToTest.nodeLink!=None):
        nodeToTest=nodeToTest.nodeLink
    nodeToTest.nodeLink=targetNode
    #print(frequentItems)
    #print(headerTable)
def createInitSet(dataSet):
    retDict={}
    for transaction in dataSet:
        retDict[frozenset(transaction)]=dataSet.count(transaction)
    return retDict

def ascendTree(leafNode,prefixPath):
    if leafNode.parent!=None:
        prefixPath.append(leafNode.name)
        ascendTree(leafNode.parent,prefixPath)
def findPrefixPath(basePat,treeNode):
    condPats={}
    while treeNode!=None:
        prefixPath=[]
        ascendTree(treeNode,prefixPath)
        if len(prefixPath)>1:
            condPats[frozenset(prefixPath[1:])]=treeNode.count   ####从1开始，去掉从自己
        treeNode=treeNode.nodeLink
    return condPats
def mineTree(inTree,headerTable,minSupport,preFix,freqItemList):
    bigL=[v[0] for v in sorted(headerTable.items(),key=lambda x: x[1][0])]
    for basePat in bigL:
        newFreqSet=preFix.copy()
        newFreqSet.add(basePat)
        freqItemList.append(newFreqSet)
        condPattBases=findPrefixPath(basePat,headerTable[basePat][1])
        myCondTree,myHead=createTree(condPattBases,minSupport)
        if myHead!=None:
            mineTree(myCondTree,myHead,minSupport,newFreqSet,freqItemList)
def fpgrowth(dataSet,minSupport):
    myfptree,myheadertable=createTree(createInitSet(dataSet),minSupport*len(dataSet))
    freqItems=[]
    mineTree(myfptree,myheadertable,minSupport*len(dataSet),set([]),freqItems)
    return freqItems


testData=[]
for i in range(0,10):
    testData.append(range(0,i))
fpgrowth(testData,0.4)

[{5},
 {0, 5},
 {1, 5},
 {0, 1, 5},
 {2, 5},
 {0, 2, 5},
 {1, 2, 5},
 {0, 1, 2, 5},
 {3, 5},
 {0, 3, 5},
 {1, 3, 5},
 {0, 1, 3, 5},
 {2, 3, 5},
 {0, 2, 3, 5},
 {1, 2, 3, 5},
 {0, 1, 2, 3, 5},
 {4, 5},
 {0, 4, 5},
 {1, 4, 5},
 {0, 1, 4, 5},
 {2, 4, 5},
 {0, 2, 4, 5},
 {1, 2, 4, 5},
 {0, 1, 2, 4, 5},
 {3, 4, 5},
 {0, 3, 4, 5},
 {1, 3, 4, 5},
 {0, 1, 3, 4, 5},
 {2, 3, 4, 5},
 {0, 2, 3, 4, 5},
 {1, 2, 3, 4, 5},
 {0, 1, 2, 3, 4, 5},
 {4},
 {0, 4},
 {1, 4},
 {0, 1, 4},
 {2, 4},
 {0, 2, 4},
 {1, 2, 4},
 {0, 1, 2, 4},
 {3, 4},
 {0, 3, 4},
 {1, 3, 4},
 {0, 1, 3, 4},
 {2, 3, 4},
 {0, 2, 3, 4},
 {1, 2, 3, 4},
 {0, 1, 2, 3, 4},
 {3},
 {0, 3},
 {1, 3},
 {0, 1, 3},
 {2, 3},
 {0, 2, 3},
 {1, 2, 3},
 {0, 1, 2, 3},
 {2},
 {0, 2},
 {1, 2},
 {0, 1, 2},
 {1},
 {0, 1},
 {0}]